## Assignment 9.1

In [1]:
"""
import os

# setup arguments
submit_args = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.0'
os.environ['PYSPARK_SUBMIT_ARGS'] = submit_args

if 'PYSPARK_SUBMIT_ARGS' not in os.environ:
    os.environ['PYSPARK_SUBMIT_ARGS'] = submit_args
else:
    os.environ['PYSPARK_SUBMIT_ARGS'] += submit_args

# initialize spark

os.environ["PYSPARK_PYTHON"]
os.environ["PYSPARK_DRIVER_PYTHON"] = "juypter"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python"

"""
print()

In [2]:
import os
import time
submit_args = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.0'
submit_args = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.0,org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.0'
submit_args = '--packages org.apache.spark:spark-sql_2.13,org.apache.spark:spark-streaming-kafka-0-8_2.11'

submit_args = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.0,org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.0'
submit_args = '--master local[3] pyspark-shell --packages org.apache.spark:spark-sql-kafka-0-10_2.13:3.2.0,org.apache.spark:spark-streaming-kafka-0-10_2.13:3.2.0'
#submit_args = '--packages org.apache.spark:spark-sql-kafka-0-10_2.13:3.2.0,org.apache.spark:spark-streaming-kafka-0-10_2.13:3.2.0'
#submit_args = '--master local[3] pyspark-shell'

submit_args = "--master local --packages org.apache.spark:spark-streaming-kafka-0-10_2.13:3.2.1 pyspark-shell"
submit_args = "--master local --packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.1 pyspark-shell"

os.environ['PYSPARK_SUBMIT_ARGS'] = submit_args



os.environ['PYSPARK_PYTHON']="python"
#os.environ['PYSPARK_PYTHON']="./environment/bin/python"


os.environ['PYSPARK_DRIVER_PYTHON']="juypter"
#os.environ['PYSPARK_DRIVER_PYTHON']="python"
# initialize spark
#import pyspark
#import findspark
#findspark.init()

In [3]:
import os
import shutil
import json
from pathlib import Path

import pandas as pd

from kafka import KafkaProducer, KafkaAdminClient
from kafka.admin.new_topic import NewTopic
from kafka.errors import TopicAlreadyExistsError

from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark import SparkConf
from pyspark.sql.functions import window, from_json, col
from pyspark.sql.types import StringType, TimestampType, DoubleType, StructField, StructType
from pyspark.sql.functions import udf
from IPython.display import clear_output
import findspark
#findspark.init()


current_dir = Path(os.getcwd()).absolute()
checkpoint_dir = current_dir.joinpath('checkpoints')
locations_checkpoint_dir = checkpoint_dir.joinpath('locations')
accelerations_checkpoint_dir = checkpoint_dir.joinpath('accelerations')

if locations_checkpoint_dir.exists():
    shutil.rmtree(locations_checkpoint_dir)
    
if accelerations_checkpoint_dir.exists():
    shutil.rmtree(accelerations_checkpoint_dir)

locations_checkpoint_dir.mkdir(parents=True, exist_ok=True)
accelerations_checkpoint_dir.mkdir(parents=True, exist_ok=True)

In [4]:
"""
refresh_kafka = True
import time
#//*** Get Working Directory
current_dir = Path(os.getcwd()).absolute()

#//*** Go up Two folders
kafka_dir = current_dir.parents[2].joinpath("kafka_31")
tmp_dir = kafka_dir.joinpath("tmp")
log_dir = kafka_dir.joinpath("logs")
print(kafka_dir)


res = os.system(str(kafka_dir)+"\stop_kafka-server-stop.bat")
print("Stop Running Kafka Server: ", res)


res = os.system(str(kafka_dir)+"\stop_zookeeper-server-stop.bat")
print("Stop Running Zookeeper Server: ", res)
print("Waiting For everything to close gracefully.....")
time.sleep(5)

if refresh_kafka:
    #//*** Delete Kafka Log Files
    for root, dirs, files in os.walk(log_dir, topdown=False):
        #//*** Delete all the log files
        for file in files:
            os.remove(Path(root).joinpath(file))
        

    #//*** Delete Kafka Database
    for root, dirs, files in os.walk(tmp_dir, topdown=False):
        #//*** Delete all the log files
        for file in files:
            os.remove(Path(root).joinpath(file))

        for folder in dirs:
            os.rmdir(Path(root).joinpath(folder))

time.sleep(1)
print("Starting Zookeeper: ")
os.system(f"start {str(kafka_dir)}\start_Kafka_zookeeper.bat")

time.sleep(1)
print("Starting Kafka Server: ")
print(f"start {str(kafka_dir)}\start_Kafka_server.bat")
os.system(f"start {str(kafka_dir)}\start_Kafka_server.bat")
"""
print("Outboarding this...")




Outboarding this...


### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [5]:
config = dict(
    bootstrap_servers=['127.0.0.1:9092'],
    first_name='ition',
    last_name='Admin'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config['locations_topic'] = '{}-locations'.format(config['topic_prefix'])
config['accelerations_topic'] = '{}-accelerations'.format(config['topic_prefix'])
config['simple_topic'] = '{}-simple'.format(config['topic_prefix'])

config

{'bootstrap_servers': ['127.0.0.1:9092'],
 'first_name': 'ition',
 'last_name': 'Admin',
 'client_id': 'Adminition',
 'topic_prefix': 'Adminition',
 'locations_topic': 'Adminition-locations',
 'accelerations_topic': 'Adminition-accelerations',
 'simple_topic': 'Adminition-simple'}

### Create Topic Utility Function

The `create_kafka_topic` helps create a Kafka topic based on your configuration settings.  For instance, if your first name is *John* and your last name is *Doe*, `create_kafka_topic('locations')` will create a topic with the name `DoeJohn-locations`.  The function will not create the topic if it already exists. 

In [6]:
def create_kafka_topic(topic_name, config=config, num_partitions=1, replication_factor=1):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']
    name = '{}-{}'.format(topic_prefix, topic_name)
    
    admin_client = KafkaAdminClient(
        bootstrap_servers=bootstrap_servers, 
        client_id=client_id
    )
    
    topic = NewTopic(
        name=name,
        num_partitions=num_partitions,
        replication_factor=replication_factor
    )

    topic_list = [topic]
    try:
        admin_client.create_topics(new_topics=topic_list)
        print('Created topic "{}"'.format(name))
    except TopicAlreadyExistsError as e:
        print('Topic "{}" already exists'.format(name))
for topic in ['locations','accelerations']:
    create_kafka_topic(topic)

Topic "Adminition-locations" already exists
Topic "Adminition-accelerations" already exists


https://sparkbyexamples.com/pyspark/pyspark-exception-java-gateway-process-exited-before-sending-the-driver-its-port-number/

https://quabr.com/58723314/pyspark-failed-to-find-data-source-kafka

https://search.maven.org/search?q=g:org.apache.spark%20AND%20a:spark-streaming-kafka-0-8-assembly_2.11

https://www.rittmanmead.com/blog/2017/01/getting-started-with-spark-streaming-with-python-and-kafka/

https://www.analyticsvidhya.com/blog/2021/06/setting-up-real-time-structured-streaming-with-spark-and-kafka-on-windows-os/

https://spark.apache.org/docs/latest//api/python/reference/api/pyspark.SparkConf.html

https://stackoverflow.com/questions/54227744/pyspark-2-x-programmatically-adding-maven-jar-coordinates-to-spark?__cpo=aHR0cHM6Ly9zdGFja292ZXJmbG93LmNvbQ

https://duckduckgo.com/?t=ffab&q=SparkSession+readstream+Failed+to+find+data+source%3A+kafka&ia=web


reintsall pyspark: https://sparkbyexamples.com/pyspark-tutorial/#pyspark-installation

conda install -c conda-forge pyspark

https://kontext.tech/column/spark/298/get-the-current-spark-context-settingsconfigurations

Find Spark add_packages resolved the local Spark Issue.
https://github.com/minrk/findspark/pull/11

In [7]:
"""
import findspark
findspark.init()

try:
    sc.stop()
except:
    pass
try:
    spark.stop()
except:
    pass
try:
    ssc.stop()
except:
    pass


"spark-streaming-kafka-0-10_2.12"
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
conf = SparkConf()
conf.set('spark.jars.packages','org.apache.spark:spark-streaming_2.10:2.2.3',)
conf.getAll()

sc = SparkContext(conf=conf) 

ssc = StreamingContext(sc, 1)

   
#dir(sc)
"""
print()

In [8]:
try:
    sc.stop()
except:
    pass
try:
    spark.stop()
except:
    pass
try:
    ssc.stop()
except:
    pass

In [9]:
"""from pyspark.sql import SparkSession
SparkSession.getActiveSession()

configurations = spark.sparkContext.getConf().getAll()
for conf in configurations:
    print(conf)
    """

'from pyspark.sql import SparkSession\nSparkSession.getActiveSession()\n\nconfigurations = spark.sparkContext.getConf().getAll()\nfor conf in configurations:\n    print(conf)\n    '

In [10]:
try:
    del findspark
except:
    pass

import findspark
findspark.add_packages("--packages org.apache.spark:spark-streaming_2.10:2.2.3,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0")
findspark.init()


In [11]:


import os
import time
#submit_args = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.0'
#submit_args = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.0,org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.0'
#submit_args = '--master local[3] pyspark-shell --packages org.apache.spark:spark-sql_2.13,org.apache.spark:spark-streaming-kafka-0-8_2.11'
#os.environ['PYSPARK_SUBMIT_ARGS'] = submit_args

#pkg='org.apache.spark:spark-streaming-kafka_2.10:3.2.1'
#pkg = "org.apache.spark:spark-streaming-kafka-0-8-assembly_2.11:2.4.8"
#pkg= 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0'
#pkg= "--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2,org.apache.kafka:kafka-clients:2.8.1"
#pkg = "--packages org.apache.spark:spark-streaming_2.10:2.2.3"

#//*** Most Recent

#//*** command line: pyspark --packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0
pkg = "org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0"

#os.environ['PYSPARK_SUBMIT_ARGS'] = pkg
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'
#os.environ['hadoop_home'] = "C:\\hadoop\\hadoop-3.3.1"
os.environ['hadoop_home'] = "C:\\apps\\spark-3.2.1-bin-hadoop3.2"
#os.environ['spark_home'] = "C:\\apps\\spark-3.2.1-bin-hadoop3.2"
os.environ['path'] += "C:\\apps\\spark-3.2.1-bin-hadoop3.2\\bin"
print(os.environ['hadoop_home'])
print(os.environ['spark_home'])

#//*** Close Spark if already running. Guarantees Spark is loaded with current settings. 
#//*** Prevents some Ipython/Notebook related issues.
try:
    spark.stop()
except:
    pass

spark = SparkSession.builder.master("local[*]")\
        .appName("Assignment09")\
        .getOrCreate()

#.config('spark.jars.packages', pkg)\
print(config['bootstrap_servers'])
df_locations = spark.readStream.format("kafka")\
    .option("kafka.bootstrap.servers", "localhost:9092")\
    .option("subscribe", config['locations_topic'])\
    .option("startingOffsets", "earliest")\
    .option("includeHeaders", "true")\
    .load()

df = df_locations \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("console") \
  .start()

time.sleep(3)
df.status

C:\apps\spark-3.2.1-bin-hadoop3.2
C:\Users\family\anaconda3\envs\DSC650\lib\site-packages\pyspark
['127.0.0.1:9092']


{'message': 'Terminated with exception: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [12]:
#.config('spark.jars.packages', pkg)\
print(config['bootstrap_servers'])
df_locations = spark.readStream.format("kafka")\
    .option("kafka.bootstrap.servers", "localhost:9092")\
    .option("subscribe", config['locations_topic'])\
    .option("startingOffsets", "earliest")\
    .option("includeHeaders", "true")\
    .load()





['127.0.0.1:9092']


In [13]:
#def foreach_batch_function(df, epoch_id):
#    print(epoch_id,df)
#df_locations.writeStream.foreachBatch(foreach_batch_function).start()  


df = df_locations \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("console") \
  .start().awaitTermination()


StreamingQueryException: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
=== Streaming Query ===
Identifier: [id = 63838e7a-4816-46ba-bd39-9fa5e7c9e835, runId = c50d93cd-aa61-491f-b21a-c9c7f2d677d4]
Current Committed Offsets: {}
Current Available Offsets: {}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
WriteToMicroBatchDataSource ConsoleWriter[numRows=20, truncate=true]
+- Project [cast(key#38 as string) AS key#54, cast(value#39 as string) AS value#55]
   +- StreamingDataSourceV2Relation [key#38, value#39, topic#40, partition#41, offset#42L, timestamp#43, timestampType#44, headers#45], org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaScan@5067a66d, KafkaV2[Subscribe[Adminition-locations]]


In [ ]:
df


In [ ]:
print(type(df))
dir(df)
print(df.exception())

In [ ]:
df.stop()

In [ ]:
dir(df)

In [ ]:
#df = df_locations.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)", "timestamp")

#df.writeStream.format("console").outputMode("Update").start()


https://sparkbyexamples.com/pyspark-tutorial/

watermarks are discussed here:
https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html

In [ ]:
#df_locations.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
#df = df_locations.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)", "timestamp")
#df.printSchema()
#df.isStreaming
#type(df)
#print(df.show())
#stream_detail_df.printSchema()
#stream_detail_df.writeStream.format("console").outputMode("Complete").start()

**TODO:** Create a data frame called `df_accelerations` that reads from the accelerations topic you published to in assignment 8.  In order to read data from this topic, make sure that you are running the notebook you created in assignment 8 that publishes acceleration and location data to the `LastnameFirstname-simple` topic. 

In [ ]:
df_accelerations = spark.readStream.format("kafka")\
    .option("kafka.bootstrap.servers", "localhost:9092")\
    .option("subscribe", config['accelerations_topic'])\
    .option("startingOffsets", "earliest")\
    .option("includeHeaders", "true")\
    .load()
    

**TODO:** Create two streaming queries, `ds_locations` and `ds_accelerations` that publish to the `LastnameFirstname-simple` topic. See http://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#starting-streaming-queries and http://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html for more information. 

In [ ]:
ds_locations = ''

ds_accelerations = ''

try:
    ds_locations.awaitTermination()
    ds_accelerations.awaitTermination()
except KeyboardInterrupt:
    print("STOPPING STREAMING DATA")